# 🚀 02_07_ElectricalSystems_Data_Generation.ipynb

This notebook generates **synthetic electrical systems data** for the Databricks AeroDemo pipeline.
It creates component-level metrics focused on the aircraft’s electrical health, including power delivery and system integrity.

---

### 📋 What this notebook does:

✅ **Sets up aircraft fleet**
- Uses the same fleet IDs as earlier generators (`A320`, `B737`, `A330`)

✅ **Generates daily electrical system records**
- For each aircraft:
  - Simulates **battery voltage** and **charge level**
  - Includes **power distribution efficiency** and **electrical load**
  - Captures **transformer + converter health** and fault codes
  - Randomly injects warnings or anomalies for realistic stress testing

✅ **Creates a structured dataset**
- Component records → CSV for Auto Loader (`/Volumes/arao/aerodemo/tmp/electrical`)

✅ **Saves output safely**
- Ensures old sample files are cleaned before saving new ones

---

### 🛠 Key points:
- **Focus area:** Electrical power system status and integrity
- **Data diversity:** Includes normal operations, load variations, and fault signals
- **Downstream integration:** Feeds DLT tables like `twin_electrical_systems` and anomaly detection workflows

---

### 🔗 Where this fits:
This notebook is part of the **02_ series** synthetic data generators:
- `02_01_Sensor_Data_Generation.ipynb`
- `02_02_Engine_Data_Generation.ipynb`
- `02_03_CabinPressurization_Data_Generation.ipynb`
- `02_04_Airframe_Synthetic_Data_Generation.ipynb`
- `02_05_LandingGear_Data_Generation.ipynb`
- `02_06_Avionics_Data_Generation.ipynb`
- `02_07_ElectricalSystems_Data_Generation.ipynb` (this file)
- Future: additional systems like fuel, hydraulics, environmental

In [0]:
import os
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import random

def generate_electrical_systems_data(num_records_per_aircraft=100):
    aircraft_ids = ["A320_101", "A330_201", "B737_301"]
    data = {
        'electrical_id': [],
        'aircraft_id': [],
        'event_timestamp': [],
        'battery_voltage': [],
        'charge_level': [],
        'power_distribution_efficiency': [],
        'electrical_load': [],
        'transformer_health': [],
        'fault_code': []
    }

    for aircraft_id in aircraft_ids:
        for i in range(num_records_per_aircraft):
            random_days_ago = random.randint(0, 6)
            random_time = datetime.now() - timedelta(
                days=random_days_ago,
                hours=random.randint(0, 23),
                minutes=random.randint(0, 59)
            )
            data['electrical_id'].append(f'EL_{aircraft_id}_{i:03d}')
            data['aircraft_id'].append(aircraft_id)
            data['event_timestamp'].append(random_time.strftime("%Y-%m-%d %H:%M:%S"))
            data['battery_voltage'].append(round(np.random.uniform(24, 28), 2))  # volts
            data['charge_level'].append(round(np.random.uniform(50, 100), 2))    # percentage
            data['power_distribution_efficiency'].append(round(np.random.uniform(85, 99), 2))  # percentage
            data['electrical_load'].append(round(np.random.uniform(100, 500), 2))  # amps
            data['transformer_health'].append(round(np.random.uniform(0.8, 1.0), 2))  # normalized
            data['fault_code'].append(random.choice(['NONE', 'WARN', 'FAIL']))

    df = pd.DataFrame(data)
    return df

# Generate DataFrame
df = generate_electrical_systems_data()

# Save to Auto Loader-compatible path
output_path = "/Volumes/arao/aerodemo/tmp/electrical"
os.makedirs(output_path, exist_ok=True)
output_file = f"{output_path}/electrical_systems_sample.csv"
if os.path.exists(output_file):
    os.remove(output_file)  # safely remove old file

df.to_csv(output_file, index=False)

print(f"✅ Electrical systems data generated: {len(df)} rows saved to {output_file}")